In [1]:
import os

In [2]:
%pwd

'/config/workspace/End-to-End-Machine-Learning-Project/research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
pip install -r requirements.txt

Obtaining file:///config/workspace/End-to-End-Machine-Learning-Project (from -r requirements.txt (line 13))
     |████████████████████████████████| 4.0 MB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 11.1 MB 38.6 MB/s eta 0:00:01
     |████████████████████████████████| 9.2 MB 55.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 55.9 MB/s eta 0:00:01
     |████████████████████████████████| 736 kB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 59.8 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 9.2 MB 56.2 MB/s eta 0:00:01
     |████████████████████████████████| 380 kB 57.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 54.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 50.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 53.4 MB/s eta 0:0

In [12]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-15 02:37:16,227:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-01-15 02:37:16,231:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-15 02:37:16,240:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-01-15 02:37:16,242:INFO:common:created directory at: artifacts]
[2024-01-15 02:37:16,243:INFO:common:created directory at: artifacts/data_ingestion]
[2024-01-15 02:37:16,911:INFO:3447005916:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26156
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "451272bc2018e4e4a4007b5a7a2b18091175f408051446cd2356b9dc40fb36b0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B5F2:219118:AADADF:BE3794:65A44D04
Accept-Ranges: bytes
Date: Sun, 14 Jan 2024 21:07:16